In [3]:
# Depression Detection with Proper Emoji Integration (Fixed Version)

import pandas as pd
import numpy as np
import re
import emoji
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Input, Concatenate, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, TFBertModel
import gc
from tqdm import tqdm

# Configuration
tf.keras.mixed_precision.set_global_policy('mixed_float16')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

np.random.seed(42)
tf.random.set_seed(42)

class DepressionDetector:
    def __init__(self):
        self.emoji_to_score = None
        self.model = None
        self.tokenizer = None
        self.max_length = 128

    def load_data(self):
        emoji_scores = pd.read_csv('/kaggle/input/emoji-sentiment/emoji_sentiment_score.csv')
        self.emoji_to_score = dict(zip(emoji_scores['Char'], emoji_scores['Sentiment score']))

        # Load and balance datasets
        dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_d_tweets.csv', usecols=['tweet']).assign(label=1)
        non_dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_non_d_tweets.csv', usecols=['tweet']).assign(label=0)
        non_dep = non_dep.sample(n=len(dep), random_state=42)

        # Load sarcasm data
        sarcasm = pd.concat([chunk[chunk.label == 1] for chunk in
                           pd.read_csv('/kaggle/input/sarcasm/train-balanced-sarcasm.csv', chunksize=10000)])
        sarcasm = sarcasm.sample(10000).assign(label=0)

        self.df = pd.concat([
            dep.rename(columns={'tweet': 'text'}),
            non_dep.rename(columns={'tweet': 'text'}),
            sarcasm.rename(columns={'comment': 'text'})
        ]).sample(frac=1).reset_index(drop=True)  # Reset index for consistency

        # Preprocess
        self.df['processed'] = self.df.text.apply(self.clean_text)
        self.df['emoji_score'] = self.df.processed.apply(self.get_emoji_score)
        self.df['bert_text'] = self.df.processed.apply(emoji.demojize)

    def clean_text(self, text):
        text = re.sub(r'http\S+|@\w+|#', '', str(text))
        return re.sub(r'\s+', ' ', text).strip()

    def get_emoji_score(self, text):
        emojis = [c for c in text if c in emoji.EMOJI_DATA]
        if not emojis: return 0.5
        return np.mean([self.emoji_to_score.get(e, 0.5) for e in emojis])

    def extract_bert_features(self):
        os.makedirs('bert_features', exist_ok=True)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert_model = TFBertModel.from_pretrained('bert-base-uncased')

        for i in tqdm(range(0, len(self.df), 16)):
            batch = self.df.bert_text.iloc[i:i+16].tolist()
            inputs = self.tokenizer(batch, return_tensors='tf', padding='max_length',
                                   truncation=True, max_length=self.max_length)
            outputs = bert_model(inputs)

            for j, idx in enumerate(range(i, min(i+16, len(self.df)))):  # Use actual indices
                np.save(f"bert_features/pooled_{idx}.npy", outputs.pooler_output[j].numpy())
                np.savez_compressed(f"bert_features/sequence_{idx}.npy", outputs.last_hidden_state[j].numpy())

        del bert_model
        gc.collect()

    def build_model(self):
        # Text processing branch
        sequence_input = Input(shape=(self.max_length, 768), dtype=tf.float16)
        lstm = Bidirectional(LSTM(128, return_sequences=True))(sequence_input)
        attention = Attention()([lstm, lstm])
        text_features = Dense(128)(attention[:, -1, :])

        # Emoji processing branch
        emoji_input = Input(shape=(1,), dtype=tf.float32)
        emoji_features = Dense(64, activation='relu')(emoji_input)
        emoji_features = Dense(32, activation='relu')(emoji_features)

        # Combined features
        combined = Concatenate()([text_features, emoji_features])
        combined = Dense(128, activation='relu')(combined)
        output = Dense(1, activation='sigmoid')(combined)

        self.model = Model(inputs=[sequence_input, emoji_input], outputs=output)
        self.model.compile(
            optimizer=tf.keras.mixed_precision.LossScaleOptimizer(Adam(3e-5)),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, indices, df, batch_size=32):
        """
        Modified to receive the entire dataframe instead of separate components
        """
        self.indices = indices
        self.df = df
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        # Calculate start and end indices for this batch
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, len(self.indices))
        
        # Get actual indices for this batch
        actual_indices = self.indices[start_idx:end_idx]
        
        # Load BERT sequence features
        seq_features = np.stack([np.load(f"bert_features/sequence_{i}.npy.npz")['arr_0'] for i in actual_indices])
        
        # Get emoji features directly from dataframe using the same indices
        emoji_features = self.df.iloc[actual_indices].emoji_score.values.reshape(0, 1)
        
        # Get labels
        labels = self.df.iloc[actual_indices].label.values
        
        return (seq_features.astype('float16'), emoji_features.astype('float32')), labels

    @property
    def output_signature(self):
        return (
            (
                tf.TensorSpec(shape=(None, 128, 768), dtype=tf.float16),
                tf.TensorSpec(shape=(None, 1), dtype=tf.float32)
            ),
            tf.TensorSpec(shape=(None,), dtype=tf.float32)
        )

def train_model():
    detector = DepressionDetector()
    detector.load_data()
    detector.extract_bert_features()
    detector.build_model()

    # Split data by index number, not by dataframe index values
    indices = np.arange(len(detector.df))
    train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)
    train_indices, val_indices = train_test_split(train_indices, test_size=0.15, random_state=42)

    # Create generators with proper output signatures, passing the full dataframe
    train_gen = DataGenerator(train_indices, detector.df)
    val_gen = DataGenerator(val_indices, detector.df)
    test_gen = DataGenerator(test_indices, detector.df)

    detector.model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        callbacks=[
            EarlyStopping(patience=5, restore_best_weights=True),
            # Fixed file extension to .keras as required by newer versions of Keras
            ModelCheckpoint('best_model.keras', save_best_only=True)
        ]
    )

    # Evaluate
    detector.model.evaluate(test_gen)
    
    # Predictions
    all_preds = []
    all_labels = []
    
    for i in range(len(test_gen)):
        inputs, labels = test_gen[i]
        predictions = detector.model.predict(inputs)
        all_preds.append(predictions)
        all_labels.append(labels)
    
    # Combine all batches
    probs = np.concatenate(all_preds).flatten()
    true_labels = np.concatenate(all_labels)
    
    print(f"ROC AUC: {roc_auc_score(true_labels, probs):.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, probs > 0.5))

class Predictor:
    def __init__(self, model_path='best_model.keras'):  # Updated file extension here too
        self.model = tf.keras.models.load_model(model_path)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = TFBertModel.from_pretrained('bert-base-uncased')
        self.emoji_scores = pd.read_csv('/kaggle/input/emoji-sentiment/emoji_sentiment_score.csv').set_index('Char')['Sentiment score']

    def predict(self, text):
        # Preprocess text
        clean_text = re.sub(r'http\S+|@\w+|#', '', text)

        # Get emoji features
        emojis = [c for c in clean_text if c in emoji.EMOJI_DATA]
        emoji_score = np.mean([self.emoji_scores.get(e, 0.5) for e in emojis]) if emojis else 0.5

        # Get BERT features
        inputs = self.tokenizer(
            emoji.demojize(clean_text),
            return_tensors='tf', 
            padding='max_length',
            truncation=True, 
            max_length=128
        )
        seq_output = self.bert_model(inputs).last_hidden_state[0].numpy().astype('float16')

        # Make prediction
        prediction = self.model.predict([seq_output[np.newaxis,...], np.array([[emoji_score]])])[0][0]

        return {
            'score': float(prediction),
            'emoji_contribution': float(prediction - (0.7*prediction + 0.3*emoji_score)),
            'text_analysis': clean_text,
            'diagnosis': 'Depressed' if prediction > 0.5 else 'Not Depressed'
        }

# Execute training and prediction
if __name__ == "__main__":
    train_model()

    # Example prediction
    predictor = Predictor()
    sample_text = "I can't handle this pain anymore 😭 Everything feels meaningless..."
    result = predictor.predict(sample_text)
    print("\n=== Depression Analysis ===")
    print(f"Text: {result['text_analysis']}")
    print(f"Depression Score: {result['score']:.4f}")
    print(f"Emoji Contribution: {result['emoji_contribution']:.4f}")
    print(f"Diagnosis: {result['diagnosis']}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


344/344 ━━━━━━━━━━━━━━━━━━━━ 49s 128ms/step - accuracy: 0.7893 - loss: 0.4235 - val_accuracy: 0.9289 - val_loss: 0.1748
Epoch 2/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 126ms/step - accuracy: 0.9368 - loss: 0.1619 - val_accuracy: 0.9304 - val_loss: 0.1684
Epoch 3/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 126ms/step - accuracy: 0.9478 - loss: 0.1346 - val_accuracy: 0.9361 - val_loss: 0.1593
Epoch 4/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 126ms/step - accuracy: 0.9577 - loss: 0.1119 - val_accuracy: 0.9356 - val_loss: 0.1527
Epoch 5/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 125ms/step - accuracy: 0.9616 - loss: 0.1044 - val_accuracy: 0.9387 - val_loss: 0.1531
Epoch 6/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 127ms/step - accuracy: 0.9647 - loss: 0.1010 - val_accuracy: 0.9371 - val_loss: 0.1684
Epoch 7/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 45s 127ms/step - accuracy: 0.9693 - loss: 0.0838 - val_accuracy: 0.9387 - val_loss: 0.1489
Epoch 8/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 127ms/step - accuracy: 0.9769 - loss: 0.0720 - val

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step

=== Depression Analysis ===
Text: I can't handle this pain anymore 😭 Everything feels meaningless...
Depression Score: 0.3015
Emoji Contribution: -0.0735
Diagnosis: Not Depressed


In [6]:
predictor.predict("I am very suicidal 😭😭")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


{'score': 0.092041015625,
 'emoji_contribution': -0.13633769531249998,
 'text_analysis': 'I am very suicidal 😭😭',
 'diagnosis': 'Not Depressed'}

In [7]:
predictor.predict("I am very suicidal 😂😂")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


{'score': 0.0250396728515625,
 'emoji_contribution': -0.10933809814453122,
 'text_analysis': 'I am very suicidal 😂😂',
 'diagnosis': 'Not Depressed'}

In [15]:
predictor.predict("😭😭")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


{'score': 0.003662109375,
 'emoji_contribution': -0.16285136718749998,
 'text_analysis': '😭😭',
 'diagnosis': 'Not Depressed'}

In [ ]:
import pandas as pd
import numpy as np
import re
import emoji
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Input, Concatenate, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, TFBertModel
import gc
from tqdm import tqdm

# Configuration
tf.keras.mixed_precision.set_global_policy('mixed_float16')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

np.random.seed(42)
tf.random.set_seed(42)

class DepressionDetector:
    def __init__(self):
        self.emoji_to_score = None
        self.model = None
        self.tokenizer = None
        self.max_length = 128

    def load_data(self):
        emoji_scores = pd.read_csv('/kaggle/input/emoji-sentiment/emoji_sentiment_score.csv')
        self.emoji_to_score = dict(zip(emoji_scores['Char'], emoji_scores['Sentiment score']))

        # Load and balance datasets
        dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_d_tweets.csv', usecols=['tweet']).assign(label=1)
        non_dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_non_d_tweets.csv', usecols=['tweet']).assign(label=0)
        non_dep = non_dep.sample(n=len(dep), random_state=42)

        # Load sarcasm data
        sarcasm = pd.concat([chunk[chunk.label == 1] for chunk in
                           pd.read_csv('/kaggle/input/sarcasm/train-balanced-sarcasm.csv', chunksize=10000)])
        sarcasm = sarcasm.sample(10000).assign(label=0)

        self.df = pd.concat([
            dep.rename(columns={'tweet': 'text'}),
            non_dep.rename(columns={'tweet': 'text'}),
            sarcasm.rename(columns={'comment': 'text'})
        ]).sample(frac=1).reset_index(drop=True)  # Reset index for consistency

        # Preprocess
        self.df['processed'] = self.df.text.apply(self.clean_text)
        self.df['emoji_score'] = self.df.processed.apply(self.get_emoji_score)
        self.df['bert_text'] = self.df.processed.apply(emoji.demojize)

    def clean_text(self, text):
        text = re.sub(r'http\S+|@\w+|#', '', str(text))
        return re.sub(r'\s+', ' ', text).strip()

    def get_emoji_score(self, text):
        emojis = [c for c in text if c in emoji.EMOJI_DATA]
        if not emojis: return 0.5
        return np.mean([self.emoji_to_score.get(e, 0.5) for e in emojis])

    def extract_bert_features(self):
        os.makedirs('bert_features', exist_ok=True)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert_model = TFBertModel.from_pretrained('bert-base-uncased')

        for i in tqdm(range(0, len(self.df), 16)):
            batch = self.df.bert_text.iloc[i:i+16].tolist()
            inputs = self.tokenizer(batch, return_tensors='tf', padding='max_length',
                                   truncation=True, max_length=self.max_length)
            outputs = bert_model(inputs)

            for j, idx in enumerate(range(i, min(i+16, len(self.df)))):  # Use actual indices
                np.save(f"bert_features/pooled_{idx}.npy", outputs.pooler_output[j].numpy())
                np.savez_compressed(f"bert_features/sequence_{idx}.npy", outputs.last_hidden_state[j].numpy())

        del bert_model
        gc.collect()

    def build_model(self):
        # Text processing branch
        sequence_input = Input(shape=(self.max_length, 768), dtype=tf.float16)
        lstm = Bidirectional(LSTM(128, return_sequences=True))(sequence_input)
        attention = Attention()([lstm, lstm])
        text_features = Dense(128)(attention[:, -1, :])

        # Emoji processing branch
        emoji_input = Input(shape=(1,), dtype=tf.float32)
        emoji_features = Dense(64, activation='relu')(emoji_input)
        emoji_features = Dense(32, activation='relu')(emoji_features)

        # Combined features
        combined = Concatenate()([text_features, emoji_features])
        combined = Dense(128, activation='relu')(combined)
        output = Dense(1, activation='sigmoid')(combined)

        self.model = Model(inputs=[sequence_input, emoji_input], outputs=output)
        self.model.compile(
            optimizer=tf.keras.mixed_precision.LossScaleOptimizer(Adam(3e-5)),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        # Save the model architecture for later recreation
        with open('model_architecture.json', 'w') as f:
            f.write(self.model.to_json())

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, indices, df, batch_size=32):
        """
        Modified to receive the entire dataframe instead of separate components
        """
        self.indices = indices
        self.df = df
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        # Calculate start and end indices for this batch
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, len(self.indices))
        
        # Get actual indices for this batch
        actual_indices = self.indices[start_idx:end_idx]
        
        # Load BERT sequence features
        seq_features = np.stack([np.load(f"bert_features/sequence_{i}.npy.npz")['arr_0'] for i in actual_indices])
        
        # Get emoji features directly from dataframe using the same indices
        emoji_features = self.df.iloc[actual_indices].emoji_score.values.reshape(-1, 1)
        
        # Get labels
        labels = self.df.iloc[actual_indices].label.values
        
        return (seq_features.astype('float16'), emoji_features.astype('float32')), labels

    @property
    def output_signature(self):
        return (
            (
                tf.TensorSpec(shape=(None, 128, 768), dtype=tf.float16),
                tf.TensorSpec(shape=(None, 1), dtype=tf.float32)
            ),
            tf.TensorSpec(shape=(None,), dtype=tf.float32)
        )

def train_model():
    detector = DepressionDetector()
    detector.load_data()
    detector.extract_bert_features()
    detector.build_model()

    # Split data by index number, not by dataframe index values
    indices = np.arange(len(detector.df))
    train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)
    train_indices, val_indices = train_test_split(train_indices, test_size=0.15, random_state=42)

    # Create generators with proper output signatures, passing the full dataframe
    train_gen = DataGenerator(train_indices, detector.df)
    val_gen = DataGenerator(val_indices, detector.df)
    test_gen = DataGenerator(test_indices, detector.df)

    # Fix: Changed ModelCheckpoint filepath to end with .weights.h5
    detector.model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        callbacks=[
            EarlyStopping(patience=5, restore_best_weights=True),
            ModelCheckpoint('best_model.weights.h5', save_best_only=True, save_weights_only=True)
        ]
    )

    # Save final weights - also fixing this filename
    detector.model.save_weights('final_model.weights.h5')

    # Evaluate
    detector.model.evaluate(test_gen)
    
    # Predictions
    all_preds = []
    all_labels = []
    
    for i in range(len(test_gen)):
        inputs, labels = test_gen[i]
        predictions = detector.model.predict(inputs)
        all_preds.append(predictions)
        all_labels.append(labels)
    
    # Combine all batches
    probs = np.concatenate(all_preds).flatten()
    true_labels = np.concatenate(all_labels)
    
    print(f"ROC AUC: {roc_auc_score(true_labels, probs):.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, probs > 0.5))
    
    # Save metadata required for prediction
    metadata = {
        'max_length': detector.max_length,
        'emoji_to_score': detector.emoji_to_score
    }
    import pickle
    with open('model_metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)

class Predictor:
    def __init__(self, weights_path='best_model.weights.h5', metadata_path='model_metadata.pkl', architecture_path='model_architecture.json'):
        # Load metadata
        import pickle
        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)
        
        self.max_length = metadata['max_length']
        
        # Initialize tokenizer and BERT model
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = TFBertModel.from_pretrained('bert-base-uncased')
        
        # Load emoji scores
        self.emoji_scores = pd.read_csv('emoji_sentiment_score.csv').set_index('Char')['Sentiment score']
        
        # Recreate model architecture
        with open(architecture_path, 'r') as f:
            model_json = f.read()
        
        self.model = tf.keras.models.model_from_json(model_json)
        
        # Load weights
        self.model.load_weights(weights_path)
        
        # Compile model
        self.model.compile(
            optimizer=tf.keras.mixed_precision.LossScaleOptimizer(Adam(3e-5)),
            loss='binary_crossentropy', 
            metrics=['accuracy']
        )

    def predict(self, text):
        # Preprocess text
        clean_text = re.sub(r'http\S+|@\w+|#', '', text)

        # Get emoji features and analyze emojis
        emojis = [c for c in clean_text if c in emoji.EMOJI_DATA]
        emoji_score = np.mean([self.emoji_scores.get(e, 0.5) for e in emojis]) if emojis else 0.5
        
        # Get BERT features for text analysis (without emojis)
        demojized_text = emoji.demojize(clean_text)
        inputs = self.tokenizer(
            demojized_text,
            return_tensors='tf',
            padding='max_length',
            truncation=True,
            max_length=self.max_length
        )
        seq_output = self.bert_model(inputs).last_hidden_state[0].numpy().astype('float16')

        # Make base prediction from text content
        base_prediction = self.model.predict([seq_output[np.newaxis,...], np.array([[0.5]])])[0][0]
        
        # Make prediction with emoji influence
        full_prediction = self.model.predict([seq_output[np.newaxis,...], np.array([[emoji_score]])])[0][0]
        
        # Calculate emoji influence (0-1 scale)
        # For sad emojis (score > 0.5), emoji_influence will be higher
        # For happy emojis (score < 0.5), emoji_influence will be lower
        emoji_influence = emoji_score if emojis else 0.5
        
        # Calculate final adjusted score with proper emoji influence
        # Base on text content but adjust based on emoji sentiment
        adjusted_score = base_prediction
        if emojis:
            # If sad emojis (>0.5), increase depression score
            if emoji_score > 0.5:
                boost_factor = (emoji_score - 0.5) * 2  # Convert 0.5-1.0 range to 0.0-1.0
                adjusted_score = min(1.0, base_prediction + (1.0 - base_prediction) * boost_factor * 0.5)
            # If happy emojis (<0.5), decrease depression score
            else:
                reduction_factor = (0.5 - emoji_score) * 2  # Convert 0.0-0.5 range to 0.0-1.0
                adjusted_score = max(0.0, base_prediction - base_prediction * reduction_factor * 0.5)

        return {
            'base_score': float(base_prediction),  # Score without emoji influence
            'emoji_score': float(emoji_score),     # Raw emoji sentiment score (0-1)
            'adjusted_score': float(adjusted_score),  # Final score with emoji adjustment
            'emoji_influence': float(emoji_influence),  # How much emojis affected the score (0-1)
            'text_analysis': clean_text,
            'diagnosis': 'Depressed' if adjusted_score > 0.5 else 'Not Depressed'
        }

# Execute training and prediction
if __name__ == "__main__":
    try:
        train_model()
        
        # Wait a moment to ensure files are properly saved
        import time
        time.sleep(1)
        
        # Example prediction
        predictor = Predictor()
        
        # Test with different emoji variations
        texts = [
            "I am very suicidal",
            "I am very suicidal 😭😭",
            "I am very suicidal 😂😂",
            "I feel so empty inside 😔",
            "Everything is going great! 🎉",
            "I can't handle this pain anymore"
        ]
        
        print("\n=== Depression Analysis Results ===")
        for test_text in texts:
            result = predictor.predict(test_text)
            print(f"\nText: {result['text_analysis']}")
            print(f"Base Score (text only): {result['base_score']:.4f}")
            print(f"Emoji Score: {result['emoji_score']:.4f}")
            print(f"Emoji Influence: {result['emoji_influence']:.4f}")
            print(f"Adjusted Score: {result['adjusted_score']:.4f}")
            print(f"Diagnosis: {result['diagnosis']}")
        
    except Exception as e:
        print(f"Error encountered: {str(e)}")
        import traceback
        traceback.print_exc()

BHAIYA iss code hath mat laganaa yeh cahl rha hai (BELOWWWW)

In [18]:
import pandas as pd
import numpy as np
import re
import emoji
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Input, Concatenate, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel
import gc
import pickle
from tqdm import tqdm

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

np.random.seed(42)
tf.random.set_seed(42)

class DepressionDetector:
    def __init__(self):
        self.emoji_to_score = None
        self.model = None
        self.tokenizer = None
        self.max_length = 128

    def load_data(self):
        emoji_scores = pd.read_csv('/kaggle/input/emoji-sentiment/emoji_sentiment_score.csv')
        self.emoji_to_score = dict(zip(emoji_scores['Char'], emoji_scores['Sentiment score']))
        dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_d_tweets.csv', usecols=['tweet']).assign(label=1)
        non_dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_non_d_tweets.csv', usecols=['tweet']).assign(label=0)
        non_dep = non_dep.sample(n=len(dep), random_state=42)
        sarcasm = pd.concat([chunk[chunk.label == 1] for chunk in pd.read_csv('/kaggle/input/sarcasm/train-balanced-sarcasm.csv', chunksize=10000)])
        sarcasm = sarcasm.sample(10000).assign(label=0)
        self.df = pd.concat([
            dep.rename(columns={'tweet': 'text'}),
            non_dep.rename(columns={'tweet': 'text'}),
            sarcasm.rename(columns={'comment': 'text'})
        ]).sample(frac=1).reset_index(drop=True)
        self.df['processed'] = self.df.text.apply(self.clean_text)
        self.df['emoji_score'] = self.df.processed.apply(self.get_emoji_score)
        self.df['bert_text'] = self.df.processed.apply(emoji.demojize)

    def clean_text(self, text):
        text = re.sub(r'http\S+|@\w+|#', '', str(text))
        return re.sub(r'\s+', ' ', text).strip()

    def get_emoji_score(self, text):
        emojis = [c for c in text if c in emoji.EMOJI_DATA]
        return np.mean([self.emoji_to_score.get(e, 0.5) for e in emojis]) if emojis else 0.5

    def extract_bert_features(self):
        os.makedirs('bert_features', exist_ok=True)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert_model = TFBertModel.from_pretrained('bert-base-uncased')
        for i in tqdm(range(0, len(self.df), 16)):
            batch = self.df.bert_text.iloc[i:i+16].tolist()
            inputs = self.tokenizer(batch, return_tensors='tf', padding='max_length', truncation=True, max_length=self.max_length)
            outputs = bert_model(inputs)
            for j, idx in enumerate(range(i, min(i+16, len(self.df)))):
                np.save(f"bert_features/pooled_{idx}.npy", outputs.pooler_output[j].numpy())
                np.savez_compressed(f"bert_features/sequence_{idx}.npy", outputs.last_hidden_state[j].numpy())
        del bert_model
        gc.collect()

    def build_model(self):
        sequence_input = Input(shape=(self.max_length, 768), dtype=tf.float16)
        lstm = Bidirectional(LSTM(128, return_sequences=True))(sequence_input)
        attention = Attention()([lstm, lstm])
        text_features = Dense(128)(attention[:, -1, :])
        emoji_input = Input(shape=(1,), dtype=tf.float32)
        emoji_features = Dense(64, activation='relu')(emoji_input)
        emoji_features = Dense(32, activation='relu')(emoji_features)
        combined = Concatenate()([text_features, emoji_features])
        combined = Dense(128, activation='relu')(combined)
        output = Dense(1, activation='sigmoid')(combined)
        self.model = Model(inputs=[sequence_input, emoji_input], outputs=output)
        self.model.compile(optimizer=tf.keras.mixed_precision.LossScaleOptimizer(Adam(3e-5)), loss='binary_crossentropy', metrics=['accuracy'])
        with open('model_architecture.json', 'w') as f:
            f.write(self.model.to_json())

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, indices, df, batch_size=32):
        self.indices = indices
        self.df = df
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = min((idx + 1) * self.batch_size, len(self.indices))
        actual_indices = self.indices[start_idx:end_idx]
        seq_features = np.stack([np.load(f"bert_features/sequence_{i}.npy.npz")['arr_0'] for i in actual_indices])
        emoji_features = self.df.iloc[actual_indices].emoji_score.values.reshape(-1, 1)
        labels = self.df.iloc[actual_indices].label.values
        return (seq_features.astype('float16'), emoji_features.astype('float32')), labels

def train_model():
    detector = DepressionDetector()
    detector.load_data()
    detector.extract_bert_features()
    detector.build_model()
    indices = np.arange(len(detector.df))
    train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)
    train_indices, val_indices = train_test_split(train_indices, test_size=0.15, random_state=42)
    train_gen = DataGenerator(train_indices, detector.df)
    val_gen = DataGenerator(val_indices, detector.df)
    test_gen = DataGenerator(test_indices, detector.df)
    detector.model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        callbacks=[
            EarlyStopping(patience=5, restore_best_weights=True),
            ModelCheckpoint('best_model.weights.h5', save_best_only=True, save_weights_only=True)
        ]
    )
    detector.model.save_weights('final_model.weights.h5')
    detector.model.evaluate(test_gen)
    all_preds, all_labels = [], []
    for i in range(len(test_gen)):
        inputs, labels = test_gen[i]
        predictions = detector.model.predict(inputs)
        all_preds.append(predictions)
        all_labels.append(labels)
    probs = np.concatenate(all_preds).flatten()
    true_labels = np.concatenate(all_labels)
    print(f"ROC AUC: {roc_auc_score(true_labels, probs):.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, probs > 0.5))
    metadata = {'max_length': detector.max_length, 'emoji_to_score': detector.emoji_to_score}
    with open('model_metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)

class Predictor:
    def __init__(self, weights_path='best_model.weights.h5', metadata_path='model_metadata.pkl', architecture_path='model_architecture.json'):
        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)
        self.max_length = metadata['max_length']
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = TFBertModel.from_pretrained('bert-base-uncased')
        self.emoji_scores = pd.read_csv('/kaggle/input/emoji-sentiment/emoji_sentiment_score.csv').set_index('Char')['Sentiment score']
        with open(architecture_path, 'r') as f:
            model_json = f.read()
        self.model = tf.keras.models.model_from_json(model_json)
        self.model.load_weights(weights_path)
        self.model.compile(optimizer=tf.keras.mixed_precision.LossScaleOptimizer(Adam(3e-5)), loss='binary_crossentropy', metrics=['accuracy'])

    def predict(self, text):
        clean_text = re.sub(r'http\S+|@\w+|#', '', text)
        emojis = [c for c in clean_text if c in emoji.EMOJI_DATA]
        text_only = ''.join(c for c in clean_text if c not in emoji.EMOJI_DATA).strip()
        emoji_score = np.mean([self.emoji_scores.get(e, 0.5) for e in emojis]) if emojis else 0.5
        demojized_text = emoji.demojize(text_only)
        
        inputs = self.tokenizer(demojized_text, return_tensors='tf', padding='max_length', truncation=True, max_length=self.max_length)
        seq_output = self.bert_model(inputs).last_hidden_state[0].numpy().astype('float16')
        base_score = self.model.predict([seq_output[np.newaxis,...], np.array([[0.5]])])[0][0]
        
        # Combine base_score with emoji sentiment influence
        final_score = (0.85 * base_score) + (0.15 * (1 - emoji_score))
        
        return {
            'original_text': text,
            'text_without_emojis': text_only,
            'emojis_found': emojis,
            'base_score': float(base_score),
            'emoji_score': float(emoji_score),
            'final_score': float(final_score),
            'diagnosis': 'Depressed' if final_score > 0.5 else 'Not Depressed'
        }

            

# Train and predict
try:
    train_model()
    import time; time.sleep(1)
    predictor = Predictor()
    texts = [
        "I am very suicidal",
        "I am very suicidal 😭😭",
        "I am very suicidal 😂😂",
        "I feel so empty inside 😔",
        "Everything is going great! 🎉",
        "I can't handle this pain anymore"
    ]
    print("\n=== Depression Analysis Results ===")
    for text in texts:
        result = predictor.predict(text)
        
        print(f"Base Score (text only): {result['base_score']:.4f}")
        print(f"Emoji Score: {result['emoji_score']:.4f}")
        print(f"Emoji Influence: {result['emoji_influence']:.4f}")
        print(f"Adjusted Score: {result['adjusted_score']:.4f}")
        print(f"Diagnosis: {result['diagnosis']}")
except Exception as e:
    print(f"Error: {e}")
    import traceback; traceback.print_exc()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


344/344 ━━━━━━━━━━━━━━━━━━━━ 49s 129ms/step - accuracy: 0.7948 - loss: 0.4150 - val_accuracy: 0.9325 - val_loss: 0.1766
Epoch 2/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 127ms/step - accuracy: 0.9334 - loss: 0.1628 - val_accuracy: 0.9366 - val_loss: 0.1602
Epoch 3/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 126ms/step - accuracy: 0.9477 - loss: 0.1287 - val_accuracy: 0.9376 - val_loss: 0.1586
Epoch 4/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 125ms/step - accuracy: 0.9543 - loss: 0.1125 - val_accuracy: 0.9412 - val_loss: 0.1486
Epoch 5/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 125ms/step - accuracy: 0.9611 - loss: 0.1012 - val_accuracy: 0.9407 - val_loss: 0.1569
Epoch 6/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 127ms/step - accuracy: 0.9689 - loss: 0.0858 - val_accuracy: 0.9443 - val_loss: 0.1426
Epoch 7/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 126ms/step - accuracy: 0.9703 - loss: 0.0822 - val_accuracy: 0.9418 - val_loss: 0.1543
Epoch 8/15
344/344 ━━━━━━━━━━━━━━━━━━━━ 44s 127ms/step - accuracy: 0.9790 - loss: 0.0650 - val

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w


=== Depression Analysis Results ===
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step
Error: 'text_analysis'


Traceback (most recent call last):
  File "/tmp/ipykernel_31/69562147.py", line 205, in <cell line: 0>
    print(f"\nText: {result['text_analysis']}")
                     ~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'text_analysis'


In [36]:

for text in texts:
        result = predictor.predict(text)
        final_score = (0.85 * result['base_score']) + (0.15 * result['emoji_score'])
        print(text)
        print(f"Base Score (text only): {result['base_score']:.4f}")
        print(f"Emoji Score: {result['emoji_score']:.4f}")
    
        print(final_score)
        
        print(f"Diagnosis: {result['diagnosis']}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
I am very suicidal
Base Score (text only): 0.9932
Emoji Score: 0.5000
0.9191894531249999
Diagnosis: Depressed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
I am very suicidal 😭😭
Base Score (text only): 0.9932
Emoji Score: 0.5465
0.926164453125
Diagnosis: Depressed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
I am very suicidal 😂😂
Base Score (text only): 0.9932
Emoji Score: 0.3895
0.9026144531249999
Diagnosis: Depressed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
I feel so empty inside 😔
Base Score (text only): 0.9971
Emoji Score: 0.5730
0.933459765625
Diagnosis: Depressed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Everything is going great! 🎉
Base Score (text only): 0.0005
Emoji Score: 0.1310
0.02010719146728516
Diagnosis: Not Depressed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
I can't handle this pain anymore
Base Score (text only): 0.9980
Emoji Score: 0.5000
0.9233398437499999
Diagnosis: Depressed


In [38]:
text = input('Enter text')
result = predictor.predict(text)
final_score = (0.85 * result['base_score']) + (0.15 * result['emoji_score'])
print(text)
print(f"Base Score (text only): {result['base_score']:.4f}")
print(f"Emoji Score: {result['emoji_score']:.4f}")

print(final_score)

print(f"Diagnosis: {result['diagnosis']}")

Enter text 😂😂


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
😂😂
Base Score (text only): 0.0149
Emoji Score: 0.3895
0.07109017639160155
Diagnosis: Not Depressed


In [ ]:
UPARRRR WALE CODE KO HAATH MAT LAGANAAA

In [31]:
import os
import re
import gc
import pickle
import emoji
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import Sequence

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

class DepressionDetector:
    def _init_(self):
        self.emoji_to_score = {}
        self.model = None
        self.tokenizer = None
        self.max_length = 128
        self.df = None

    def load_data(self):
        # Load emoji sentiment scores
        emoji_scores = pd.read_csv('/kaggle/input/emoji-sentiment/emoji_sentiment_score.csv')
        self.emoji_to_score = dict(zip(emoji_scores['Char'], emoji_scores['Sentiment score']))
        # Load tweets
        dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_d_tweets.csv', usecols=['tweet']).assign(label=1)
        non_dep = pd.read_csv('/kaggle/input/twitter-depression-dataset/clean_non_d_tweets.csv', usecols=['tweet']).assign(label=0)
        non_dep = non_dep.sample(n=len(dep), random_state=42)
        sarcasm = pd.concat(
            [chunk[chunk.label == 1] 
             for chunk in pd.read_csv('/kaggle/input/sarcasm/train-balanced-sarcasm.csv', chunksize=10000)]
        ).sample(10000).assign(label=0)
        # Combine
        self.df = pd.concat([
            dep.rename(columns={'tweet': 'text'}),
            non_dep.rename(columns={'tweet': 'text'}),
            sarcasm.rename(columns={'comment': 'text'})
        ]).sample(frac=1, random_state=42).reset_index(drop=True)
        # Preprocess
        self.df['processed'] = self.df.text.apply(self.clean_text)
        self.df['emoji_score'] = self.df.processed.apply(self.get_emoji_score)
        self.df['bert_text'] = self.df.processed.apply(emoji.demojize)

    def clean_text(self, text):
        text = re.sub(r'http\S+|@\w+|#', '', str(text))
        return re.sub(r'\s+', ' ', text).strip()

    def get_emoji_score(self, text):
        emojis = [c for c in text if c in emoji.EMOJI_DATA]
        if not emojis:
            return 0.5
        return np.mean([self.emoji_to_score.get(e, 0.5) for e in emojis])

    def extract_bert_features(self):
        os.makedirs('bert_features', exist_ok=True)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        bert_model = TFBertModel.from_pretrained('bert-base-uncased')
        for i in tqdm(range(0, len(self.df), 16)):
            batch = self.df.bert_text.iloc[i:i+16].tolist()
            inputs = self.tokenizer(batch, return_tensors='tf',
                                    padding='max_length', truncation=True,
                                    max_length=self.max_length)
            outputs = bert_model(**inputs)
            for j, idx in enumerate(range(i, min(i+16, len(self.df)))):
                # Save sequence features only
                np.savez_compressed(f"bert_features/sequence_{idx}.npz",
                                    arr=outputs.last_hidden_state[j].numpy())
        del bert_model
        gc.collect()

    def build_model(self):
        # Text sequence input
        sequence_input = Input(shape=(self.max_length, 768), dtype=tf.float16, name='text_input')
        x = Bidirectional(LSTM(128, return_sequences=True))(sequence_input)
        x = Attention()([x, x])
        text_features = Dense(128, activation='relu')(x[:, -1, :])
        # Emoji score input
        emoji_input = Input(shape=(1,), dtype=tf.float32, name='emoji_input')
        e = Dense(64, activation='relu')(emoji_input)
        e = Dense(32, activation='relu')(e)
        # Combine
        combined = Concatenate()([text_features, e])
        combined = Dense(128, activation='relu')(combined)
        output = Dense(1, activation='sigmoid', name='output')(combined)
        self.model = Model(inputs=[sequence_input, emoji_input], outputs=output)
        optimizer = Adam(3e-5)
        self.model.compile(optimizer=optimizer,
                           loss='binary_crossentropy',
                           metrics=['accuracy'])
        # Save architecture
        with open('model_architecture.json', 'w') as f:
            f.write(self.model.to_json())

class DataGenerator(Sequence):
    def _init_(self, indices, df, batch_size=32):
        self.indices = indices
        self.df = df
        self.batch_size = batch_size

    def _len_(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def _getitem_(self, idx):
        start = idx * self.batch_size
        end = min((idx + 1) * self.batch_size, len(self.indices))
        batch_idx = self.indices[start:end]
        # Load BERT sequence features
        seqs = np.stack([
            np.load(f"bert_features/sequence_{i}.npz")['arr']
            for i in batch_idx
        ])
        emojis = self.df.loc[batch_idx, 'emoji_score'].values.reshape(-1, 1)
        labels = self.df.loc[batch_idx, 'label'].values
        return [seqs.astype('float16'), emojis.astype('float32')], labels

class Predictor:
    def __init__(self,
                 weights_path='best_model.weights.h5',
                 metadata_path='model_metadata.pkl',
                 architecture_path='model_architecture.json'):
        # Load metadata
        metadata = pickle.load(open(metadata_path, 'rb'))
        self.max_length = metadata['max_length']
        self.emoji_to_score = metadata['emoji_to_score']
        # Tokenizer & BERT
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = TFBertModel.from_pretrained('bert-base-uncased')
        # Load model
        model_json = open(architecture_path, 'r').read()
        self.model = tf.keras.models.model_from_json(model_json)
        self.model.load_weights(weights_path)
        self.model.compile(optimizer=Adam(3e-5),
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

    def predict(self, text):
        # Clean & extract emojis
        clean_text = re.sub(r'http\S+|@\w+|#', '', text)
        emojis = [c for c in clean_text if c in emoji.EMOJI_DATA]
        text_only = ''.join(c for c in clean_text if c not in emoji.EMOJI_DATA).strip()
        emoji_score = np.mean([self.emoji_to_score.get(e, 0.5) for e in emojis]) if emojis else 0.5
        demojized = emoji.demojize(text_only)
        # BERT inputs
        inputs = self.tokenizer(demojized, return_tensors='tf',
                                padding='max_length', truncation=True,
                                max_length=self.max_length)
        seq_output = self.bert_model(**inputs).last_hidden_state[0].numpy().astype('float16')
        base_score = float(self.model.predict(
            [seq_output[np.newaxis, ...], np.array([[emoji_score]])]
        )[0][0])
        # Emoji influence & final
        emoji_influence = 0.15 * (1 - emoji_score)
        adjusted_score = 0.85 * base_score + emoji_influence
        diagnosis = 'Depressed' if adjusted_score > 0.5 else 'Not Depressed'
        return {
            'original_text': text,
            'base_score': base_score,
            'emoji_score': emoji_score,
            'emoji_influence': emoji_influence,
            'adjusted_score': adjusted_score,
            'diagnosis': diagnosis
        }

def train_model():
    detector = DepressionDetector()
    detector.load_data()
    detector.extract_bert_features()
    detector.build_model()

    # Split data
    idx = np.arange(len(detector.df))
    train_idx, test_idx = train_test_split(idx, test_size=0.2, random_state=42)
    train_idx, val_idx = train_test_split(train_idx, test_size=0.15, random_state=42)

    train_gen = DataGenerator(train_idx, detector.df)
    val_gen = DataGenerator(val_idx, detector.df)
    test_gen = DataGenerator(test_idx, detector.df)

    # Train
    detector.model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=15,
        callbacks=[
            EarlyStopping(patience=5, restore_best_weights=True),
            ModelCheckpoint('best_model.weights.h5', save_best_only=True, save_weights_only=True)
        ]
    )
    # Save final weights and metadata
    detector.model.save_weights('final_model.weights.h5')
    metadata = {'max_length': detector.max_length, 'emoji_to_score': detector.emoji_to_score}
    with open('model_metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)

    # Evaluate
    print("\n=== Test Evaluation ===")
    detector.model.evaluate(test_gen)
    all_preds, all_labels = [], []
    for i in range(len(test_gen)):
        (x_seq, x_emoji), y = test_gen[i]
        p = detector.model.predict([x_seq, x_emoji]).flatten()
        all_preds.append(p)
        all_labels.append(y)
    probs = np.concatenate(all_preds)
    true = np.concatenate(all_labels)
    print(f"ROC AUC: {roc_auc_score(true, probs):.4f}")
    print(classification_report(true, probs > 0.5))

if __name__ == "__main__":
    # Train and save everything
    train_model()

    # Initialize predictor
    predictor = Predictor(weights_path='best_model.weights.h5',
                          metadata_path='model_metadata.pkl',
                          architecture_path='model_architecture.json')

    # Demo texts
    demo_texts = [
        "I am very suicidal",
        "I am very suicidal 😭😭",
        "I am very suicidal 😂😂",
        "I feel so empty inside 😔",
        "Everything is going great! 🎉",
        "I can't handle this pain anymore"
    ]
    print("\n=== Demo Analysis Results ===")
    for txt in demo_texts:
        res = predictor.predict(txt)
        print(f"\nText: {res['original_text']}")
        print(f"  Base Score: {res['base_score']:.4f}")
        print(f"  Emoji Score: {res['emoji_score']:.4f}")
        print(f"  Emoji Influence: {res['emoji_influence']:.4f}")
        print(f"  Adjusted Score: {res['adjusted_score']:.4f}")
        print(f"  Diagnosis: {res['diagnosis']}")

    # Custom input
    while True:
        custom = input("\nEnter custom text (or 'exit' to quit): ")
        if custom.lower() == 'exit':
            break
        res = predictor.predict(custom)
        print(f"\nText: {res['original_text']}")
        print(f"  Base Score: {res['base_score']:.4f}")
        print(f"  Emoji Score: {res['emoji_score']:.4f}")
        print(f"  Emoji Influence: {res['emoji_influence']:.4f}")
        print(f"  Adjusted Score: {res['adjusted_score']:.4f}")
        print(f"  Diagnosis: {res['diagnosis']}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

AttributeError: 'DepressionDetector' object has no attribute 'max_length'